In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/CS238-Decision Making Under Uncertainty/rl-qed-generator-DrugForge"

import os
import sys
import subprocess
sys.path.append("/content/drive/MyDrive/CS238-Decision Making Under Uncertainty/rl-qed-generator-DrugForge/src")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CS238-Decision Making Under Uncertainty/rl-qed-generator-DrugForge


In [24]:
from smiles_data import create_smiles_dataloader

train_loader, vocab = create_smiles_dataloader(
    "data/guacamol_v1_all.smiles",
    batch_size=128,
    max_len=120,
)
for batch in train_loader:
    input_ids = batch["input_ids"]
    target_ids = batch["target_ids"]
    attn_mask = batch["attention_mask"]


In [3]:
!pip install rdkit
!pip install wandb
from smiles_data import create_smiles_dataloader
from smiles_rl_env import SmilesRLEnv
import numpy as np

train_loader, vocab = create_smiles_dataloader(
    "data/guacamol_v1_all.smiles",
    batch_size=128,
    max_len=120,
)

env = SmilesRLEnv(vocab=vocab, max_len=120, invalid_reward=0.0)

obs = env.reset()
done = False

while not done:
    action = np.random.randint(0, env.action_space_n)
    obs, reward, done, info = env.step(action)

print("Episode finished.")
print("Reward:", reward)
print("Info:", info)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 59.9 MB/s eta 0:00:00
Episode finished.
Reward: 0.0
Info: {'smiles': '5SpCFe%o092ro--Ice<UNK>]C5=l99+()icP89eFrlIP]l8b%=H27iS[r[i5s)4%#PB99FH%', 'is_valid': False, 'qed': 0.0}


[00:21:30] SMILES Parse Error: syntax error while parsing: 5SpCFe%o092ro--Ice<UNK>]C5=l99+()icP89eFrlIP]l8b%=H27iS[r[i5s)4%#PB99FH%
[00:21:30] SMILES Parse Error: check for mistakes around position 1:
[00:21:30] 5SpCFe%o092ro--Ice<UNK>]C5=l99+()icP89eFr
[00:21:30] ^
[00:21:30] SMILES Parse Error: Failed parsing SMILES '5SpCFe%o092ro--Ice<UNK>]C5=l99+()icP89eFrlIP]l8b%=H27iS[r[i5s)4%#PB99FH%' for input: '5SpCFe%o092ro--Ice<UNK>]C5=l99+()icP89eFrlIP]l8b%=H27iS[r[i5s)4%#PB99FH%'


In [26]:
import torch

from smiles_data import create_smiles_dataloader
from smiles_model import (
    SmilesRNNPolicy,
    pretrain_smiles_rnn,
    sample_smiles_batch,
    quick_validity_check,
)

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_loader, vocab = create_smiles_dataloader(
        "data/guacamol_v1_all.smiles",
        batch_size=128,
        max_len=120,
        min_freq=1,
        max_vocab_size=None,
    )

    model = pretrain_smiles_rnn(
        train_loader=train_loader,
        vocab=vocab,
        num_epochs=2,
        lr=5e-4,
        device=device,
        log_every=100,
        save_path="checkpoints/smiles_rnn_pretrained.pt",
    )

    samples = sample_smiles_batch(
        model,
        vocab,
        num_samples=10,
        max_len=120,
        greedy=False,
        device=device,
    )
    print("Sampled SMILES:")
    for s in samples:
        print(s)

    validity = quick_validity_check(
        model,
        vocab,
        num_samples=500,
        max_len=120,
        device=device,
    )
    print(f"Validity over 500 samples: {validity * 100:.2f}%")
    # assert validity > 0.90

if __name__ == "__main__":
    main()


[Epoch 1 | Step 100] Train loss: 1.3312
[Epoch 1 | Step 200] Train loss: 1.1126
[Epoch 1 | Step 300] Train loss: 1.0178
[Epoch 1 | Step 400] Train loss: 0.9312
[Epoch 1 | Step 500] Train loss: 0.9131
[Epoch 1 | Step 600] Train loss: 0.8959
[Epoch 1 | Step 700] Train loss: 0.8544
[Epoch 1 | Step 800] Train loss: 0.8601
[Epoch 1 | Step 900] Train loss: 0.8252
[Epoch 1 | Step 1000] Train loss: 0.7921
[Epoch 1 | Step 1100] Train loss: 0.7981
[Epoch 1 | Step 1200] Train loss: 0.7846
[Epoch 1 | Step 1300] Train loss: 0.7760
[Epoch 1 | Step 1400] Train loss: 0.7845
[Epoch 1 | Step 1500] Train loss: 0.7802
[Epoch 1 | Step 1600] Train loss: 0.7362
[Epoch 1 | Step 1700] Train loss: 0.7502
[Epoch 1 | Step 1800] Train loss: 0.7656
[Epoch 1 | Step 1900] Train loss: 0.7238
[Epoch 1 | Step 2000] Train loss: 0.7350
[Epoch 1 | Step 2100] Train loss: 0.7448
[Epoch 1 | Step 2200] Train loss: 0.7440
[Epoch 1 | Step 2300] Train loss: 0.7266
[Epoch 1 | Step 2400] Train loss: 0.6994
[Epoch 1 | Step 2500] Tra

[03:37:05] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 24 25 26
[03:37:05] Can't kekulize mol.  Unkekulized atoms: 2 3 14 15 20
[03:37:05] Can't kekulize mol.  Unkekulized atoms: 5 15 16 17 43 44 45
[03:37:05] Can't kekulize mol.  Unkekulized atoms: 11 12 13 17 18 19 20 37 39
[03:37:05] Can't kekulize mol.  Unkekulized atoms: 3 4 7 8 9 10 11 12 13 14 15 17 18 20 22
[03:37:05] Can't kekulize mol.  Unkekulized atoms: 2 10 18 20
[03:37:05] SMILES Parse Error: unclosed ring for input: 'COC(=O)c1ccc(Nc2cccc(Nc3c4c(nc5ccccc35)CC3)n2)cc1'
[03:37:05] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 19
[03:37:05] SMILES Parse Error: syntax error while parsing: CC(c1nc2c(C3c4c(ocnc4ncn4CC(C)O)nc3scn2C)cn1)C(c1nccs1)C([H])([H])[H]H]c1ccccc1
[03:37:05] SMILES Parse Error: check for mistakes around position 70:
[03:37:05] nccs1)C([H])([H])[H]H]c1ccccc1
[03:37:05] ~~~~~~~~~~~~~~~~~~~~^
[03:37:05] SMILES Parse Error: Failed parsing SMILES 'CC(c1nc2c(C3c4c(ocnc4ncn4CC(C)O)nc3scn2C)cn1)C(c1nccs1)C(

    Quick validity (200 samples): 87.50%
    Model checkpoint saved to checkpoints/smiles_rnn_pretrained.pt
[Epoch 2 | Step 12500] Train loss: 0.6159
[Epoch 2 | Step 12600] Train loss: 0.6168
[Epoch 2 | Step 12700] Train loss: 0.5940
[Epoch 2 | Step 12800] Train loss: 0.6017
[Epoch 2 | Step 12900] Train loss: 0.6027
[Epoch 2 | Step 13000] Train loss: 0.6185
[Epoch 2 | Step 13100] Train loss: 0.6041
[Epoch 2 | Step 13200] Train loss: 0.6007
[Epoch 2 | Step 13300] Train loss: 0.6066
[Epoch 2 | Step 13400] Train loss: 0.5980
[Epoch 2 | Step 13500] Train loss: 0.6232
[Epoch 2 | Step 13600] Train loss: 0.6048
[Epoch 2 | Step 13700] Train loss: 0.5762
[Epoch 2 | Step 13800] Train loss: 0.5843
[Epoch 2 | Step 13900] Train loss: 0.5962
[Epoch 2 | Step 14000] Train loss: 0.5650
[Epoch 2 | Step 14100] Train loss: 0.6077
[Epoch 2 | Step 14200] Train loss: 0.5885
[Epoch 2 | Step 14300] Train loss: 0.5710
[Epoch 2 | Step 14400] Train loss: 0.6008
[Epoch 2 | Step 14500] Train loss: 0.6002
[Epoch 2 |

[03:55:21] SMILES Parse Error: unclosed ring for input: 'CC1(C)Cc2nc(C3=CN4CCN5CCC3C4O)c3c(=O)cc(=O)[nH]c3c2O1'
[03:55:21] SMILES Parse Error: unclosed ring for input: 'COC(CNc1nc2c(-c3nc4cc(C)ccn4c3nccc2c2)CCCO1)C(N)=O'
[03:55:21] Can't kekulize mol.  Unkekulized atoms: 23 24 25 26 44 45 46
[03:55:21] Explicit valence for atom # 18 N, 4, is greater than permitted
[03:55:21] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 14 15 16 17 19
[03:55:21] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13 14 15
[03:55:21] SMILES Parse Error: unclosed ring for input: 'CCOc1ccc2cc(CCC3OC4OC5(C)CCC(O)C5C(C)C3CC(C)C=CC43)oc2c1O'
[03:55:21] Can't kekulize mol.  Unkekulized atoms: 8 9 10 11 23
[03:55:21] SMILES Parse Error: extra close parentheses while parsing: COCc1cc(-c2cnc(N)c(F)c2)cc2c1OCC(=O)N2CC=Cc1ccccc1)COC(=O)c1c(C)cc2ccccc2c1
[03:55:21] SMILES Parse Error: check for mistakes around position 51:
[03:55:21] CC(=O)N2CC=Cc1ccccc1)COC(=O)c1c(C)cc2cccc
[03:55:21] ~~~~~~~~~~~~~~~~~~~~^
[0

    Quick validity (200 samples): 93.00%
    Model checkpoint saved to checkpoints/smiles_rnn_pretrained.pt
Sampled SMILES:
CCOC(c1ccccc1)N1C(=O)C(C)(CC)CC1c1ccc(OC)cc1
CC(=O)N1CCC(c2cnc(Nc3ncc(C)cn3)s2)CC1
Cc1cccc(NC(=O)c2cccc(S(=O)(=O)N3CCCCCC3)c2)c1
Cc1cc(N)nc(C(CO)Oc2ccc(C(=N)N)cc2)n1
O=C1C=CC=CC1=Cc1cccc2c(C3=NCCCN3)c3ccccc3nc12
COC(=O)c1cccc(COc2ccc(C=NN(Cc3ccco3)C(=O)c3ccc(CO)cc3)cc2)c1
Cc1ccc2nc(N3CCCCCC3)nc(NCc3cccnc3)c2c1
C=C1C(O)C=C(C)C=CC(C=Cc2ccc(O)cc2)CC(O)CC1O
C=C(c1ccccc1)c1nc2ccccc2c(=O)n1N=C1CC(c2ccccc2)=NS1(N)=O
CC(C)NS(=O)(=O)c1sc(C(C)(C)C)cc1C(N)=O


[03:55:37] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 22 23
[03:55:37] Can't kekulize mol.  Unkekulized atoms: 13 14 15 16 17 18 19 21 22
[03:55:37] SMILES Parse Error: extra close parentheses while parsing: O=S(=O)(c1ccc(Cl)cc1)N1CCC2CCCN(c3ncccn3)C2)c1
[03:55:37] SMILES Parse Error: check for mistakes around position 44:
[03:55:37] CCC2CCCN(c3ncccn3)C2)c1
[03:55:37] ~~~~~~~~~~~~~~~~~~~~^
[03:55:37] SMILES Parse Error: Failed parsing SMILES 'O=S(=O)(c1ccc(Cl)cc1)N1CCC2CCCN(c3ncccn3)C2)c1' for input: 'O=S(=O)(c1ccc(Cl)cc1)N1CCC2CCCN(c3ncccn3)C2)c1'
[03:55:37] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 11
[03:55:37] SMILES Parse Error: unclosed ring for input: 'CCC(C)C1NC(=O)CCCCCCC(=O)CC(O)C(C(C)CC)C(=O)NC(Cc2ccc3ccccc3c2)C(=O)NC(C)C(N)=O'
[03:55:37] SMILES Parse Error: unclosed ring for input: 'COc1ccc(C23COC4(O)C(CCC5C4(C)CC4OC4CCOC(=O)C2(C)C)C3C=CC(C)(C)O)cc1'
[03:55:37] SMILES Parse Error: extra close parentheses while parsing: CCC(Cc1ccccc1)NC(=O)C(Cc1ccc(C#N)cc1)

Validity over 500 samples: 91.00%


In [38]:
import torch
# torch.backends.cudnn.enabled = False

import importlib
import smiles_reinforce
importlib.reload(smiles_reinforce)

from smiles_data import create_smiles_dataloader
from smiles_model import SmilesRNNPolicy
from smiles_rl_env import SmilesRLEnv
from smiles_reinforce import reinforce_train, evaluate_policy


def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_loader, vocab = create_smiles_dataloader(
        "data/guacamol_v1_all.smiles",
        batch_size=128,
        max_len=120,
        min_freq=1,
        max_vocab_size=None,
    )

    policy = SmilesRNNPolicy(
        vocab_size=len(vocab),
        embed_dim=256,
        hidden_dim=512,
        num_layers=2,
        dropout=0.1,
    )
    policy.to(device)

    pretrained_ckpt = "checkpoints/smiles_rnn_pretrained.pt"
    try:
        state_dict = torch.load(pretrained_ckpt, map_location=device)
        policy.load_state_dict(state_dict)
        print(f"Loaded pretrained weights from {pretrained_ckpt}")
    except FileNotFoundError:
        print("Warning: pretrained checkpoint not found, starting RL from scratch.")

    env = SmilesRLEnv(
        vocab=vocab,
        max_len=120,
        invalid_reward=0.0,
        qed_scale="none",
    )

    # REINFORCE
    policy = reinforce_train(
        policy=policy,
        env=env,
        num_episodes=200,
        batch_episodes=16,
        lr=1e-4,
        reward_scale=5.0,
        baseline_momentum=0.9,
        log_every=10,
        eval_every=100,
        eval_num_samples=256,
        max_len=120,
        device=device,
        save_path="checkpoints/smiles_policy_reinforce.pt",
    )

    metrics = evaluate_policy(
        policy=policy,
        vocab=vocab,
        num_samples=512,
        max_len=120,
        device=device,
    )
    print("Final policy metrics:")
    print(metrics)


if __name__ == "__main__":
    main()

Loaded pretrained weights from checkpoints/smiles_rnn_pretrained.pt


[04:13:42] Can't kekulize mol.  Unkekulized atoms: 1 2 3 31 32
[04:13:45] Can't kekulize mol.  Unkekulized atoms: 1 2 3 32 33
[04:13:54] Can't kekulize mol.  Unkekulized atoms: 20 21 22 23 24 25 27 28 29
[04:13:59] Can't kekulize mol.  Unkekulized atoms: 3 4 5 23 24
[04:14:07] SMILES Parse Error: unclosed ring for input: 'CC(C)CCCCC(C)C1CCC2C3CC(=O)C=C4CC(c5coc5ccccc45)C4(C)CCC3C4CCC12CO'
[04:14:09] Can't kekulize mol.  Unkekulized atoms: 9 10 11 26 27
[04:14:10] Can't kekulize mol.  Unkekulized atoms: 16 17 18 19 20 21 23
[04:14:32] SMILES Parse Error: unclosed ring for input: 'COCc1nccnc1-c1cc2c(Nc3ccc(CNC(=O)C4CCCN(CCC5(C)C)CC4)cc3)c3ccc(NC(=O)C4CCC(C(=O)O)CC4)cc3nc2s1'
[04:14:42] Can't kekulize mol.  Unkekulized atoms: 9 15 16 17 18
[04:14:55] Can't kekulize mol.  Unkekulized atoms: 1 2 3 16 17 18 19
[04:14:57] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 8 9 10 11 12 13 14 15 17


[Episode 10/200 | Updates 10] Avg reward (last 160 eps): 0.5675, Baseline: 2.7313, Batch loss: -0.1558


[04:15:06] SMILES Parse Error: unclosed ring for input: 'c1ccc2c(c1)CCN1CCc3ccccc3O1'
[04:15:07] SMILES Parse Error: unclosed ring for input: 'Clc1ccccc1CN1CCN(c2cc3c(cc2F)c(=O)c(=O)n4nc(CCN4CCOCC4)cn3c2CCCCCC2)CC1'
[04:15:14] SMILES Parse Error: extra close parentheses while parsing: FC(F)(F)Oc1ccc2cc3[nH]ccc3c(NCCCCNCc3ccc(CN4CCC(CN5CCCC5)CC4)cc3)c2c1)c1ccccc1
[04:15:14] SMILES Parse Error: check for mistakes around position 70:
[04:15:14] N5CCCC5)CC4)cc3)c2c1)c1ccccc1
[04:15:14] ~~~~~~~~~~~~~~~~~~~~^
[04:15:14] SMILES Parse Error: Failed parsing SMILES 'FC(F)(F)Oc1ccc2cc3[nH]ccc3c(NCCCCNCc3ccc(CN4CCC(CN5CCCC5)CC4)cc3)c2c1)c1ccccc1' for input: 'FC(F)(F)Oc1ccc2cc3[nH]ccc3c(NCCCCNCc3ccc(CN4CCC(CN5CCCC5)CC4)cc3)c2c1)c1ccccc1'
[04:15:14] Can't kekulize mol.  Unkekulized atoms: 25 26 27 28 29 30 31
[04:15:21] SMILES Parse Error: unclosed ring for input: 'O=C(Nc1ccccc1Cl)c1csc(C2CCN(C(=O)OCC34CCNCC3)CC2)n1'
[04:15:21] SMILES Parse Error: unclosed ring for input: 'NCc1cccc2c(C3=CCNCC4)c(O)c

[Episode 20/200 | Updates 20] Avg reward (last 160 eps): 0.5631, Baseline: 3.1285, Batch loss: 0.1972


...

[Episode 200/200 | Updates 200] Avg reward (last 160 eps): 0.7829, Baseline: 4.0589, Batch loss: -0.0007


[04:34:11] Can't kekulize mol.  Unkekulized atoms: 12 13 14 15 17 18 19
[04:34:12] Can't kekulize mol.  Unkekulized atoms: 17 18 19 20 21 22 23
[04:34:13] SMILES Parse Error: unclosed ring for input: 'COc1ccc(CNC(c2ccccc2)c(C)nc2Cl)cc1Cl'
[04:34:13] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6
[04:34:14] Can't kekulize mol.  Unkekulized atoms: 2 3 4 12
[04:34:15] SMILES Parse Error: unclosed ring for input: 'Cc1ccc(-n2ncc3c(-n4cncn5)ncnc3c2C)cc1'
[04:34:16] Can't kekulize mol.  Unkekulized atoms: 2 3 4 15


  [Eval @ Episode 200] Avg QED: 0.7755, Validity: 97.27%, Novelty: 98.44%
  Saved policy checkpoint to checkpoints/smiles_policy_reinforce.pt


[04:34:17] SMILES Parse Error: unclosed ring for input: 'CN1c2ccccc2NC2CN(C)Cc3ccccc31'
[04:34:18] Can't kekulize mol.  Unkekulized atoms: 2 5 6 7 8 9 10 11 12
[04:34:18] Can't kekulize mol.  Unkekulized atoms: 5 7 8 9 10
[04:34:20] Can't kekulize mol.  Unkekulized atoms: 2 11 12
[04:34:20] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 22
[04:34:20] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6
[04:34:21] Can't kekulize mol.  Unkekulized atoms: 18 19 21
[04:34:21] Can't kekulize mol.  Unkekulized atoms: 2 3 4 12
[04:34:21] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6
[04:34:23] Can't kekulize mol.  Unkekulized atoms: 11 12 13
[04:34:24] Can't kekulize mol.  Unkekulized atoms: 2 3 5 6 7 8 9 10 11
[04:34:24] SMILES Parse Error: unclosed ring for input: 'OC(CN1c2ccccc2CCc1ccccc1)c1ccccc1'
[04:34:25] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9
[04:34:26] non-ring atom 15 marked aromatic


Final policy metrics:
{'avg_qed': 0.7696308447070439, 'validity': 0.97265625, 'novelty': 0.984375}


In [39]:
import torch
# torch.backends.cudnn.enabled = False

import importlib
import smiles_PPO
importlib.reload(smiles_PPO)

from smiles_data import create_smiles_dataloader
from smiles_model import SmilesRNNPolicy
from smiles_rl_env import SmilesRLEnv
from smiles_PPO import SmilesActorCritic, ppo_train, evaluate_policy_ppo


def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_loader, vocab = create_smiles_dataloader(
        "data/guacamol_v1_all.smiles",
        batch_size=128,
        max_len=120,
        min_freq=1,
        max_vocab_size=None,
    )

    policy = SmilesRNNPolicy(
        vocab_size=len(vocab),
        embed_dim=256,
        hidden_dim=512,
        num_layers=2,
        dropout=0.1,
    ).to(device)

    pretrained_ckpt = "checkpoints/smiles_rnn_pretrained.pt"
    try:
        state_dict = torch.load(pretrained_ckpt, map_location=device)
        policy.load_state_dict(state_dict)
        print(f"Loaded pretrained weights from {pretrained_ckpt}")
    except FileNotFoundError:
        print("Warning: pretrained checkpoint not found, PPO will start from random policy.")

    # initiate Actor-Critic
    actor_critic = SmilesActorCritic.from_pretrained_policy(policy, dropout=0.1)
    actor_critic.to(device)

    env = SmilesRLEnv(
        vocab=vocab,
        max_len=120,
        invalid_reward=0.0,
        qed_scale="none",
    )

    # PPO
    actor_critic = ppo_train(
        actor_critic=actor_critic,
        env=env,
        num_updates=200,
        episodes_per_update=16,
        ppo_epochs=4,
        mini_batch_size=64,
        lr=3e-4,
        clip_range=0.2,
        value_coef=0.5,
        entropy_coef=0.01,
        max_len=120,
        temperature=1.0,
        log_every=5,
        eval_every=20,
        eval_num_samples=256,
        device=device,
        save_path="checkpoints/smiles_actor_critic_ppo.pt",
    )

    metrics = evaluate_policy_ppo(
        actor_critic,
        vocab=vocab,
        num_samples=512,
        max_len=120,
        device=device,
    )
    print("Final PPO policy metrics:")
    print(metrics)


if __name__ == "__main__":
    main()


Loaded pretrained weights from checkpoints/smiles_rnn_pretrained.pt


[04:34:39] Explicit valence for atom # 30 C, 5, is greater than permitted
[04:34:42] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 8 9 10 11 19
[04:34:44] Explicit valence for atom # 13 F, 2, is greater than permitted
[04:34:48] SMILES Parse Error: unclosed ring for input: 'c1cnn(-c2nc3c(F)ccc(Cl)c3c(-c3nnn[nH]4)nc23)n1'
[04:34:48] Can't kekulize mol.  Unkekulized atoms: 15 16 17 18 19 21 22 23 24
[04:34:49] Can't kekulize mol.  Unkekulized atoms: 15 16 22
[04:34:50] Can't kekulize mol.  Unkekulized atoms: 2 3 5 6 15 22 23
[04:34:50] SMILES Parse Error: unclosed ring for input: 'O=C1C=C(Cl)c2ccc(OCc3ccccc3)cc2C2=N1'
[04:34:53] Can't kekulize mol.  Unkekulized atoms: 25 26 27 28 31
[04:34:54] Can't kekulize mol.  Unkekulized atoms: 7 9 10 11 12 16 17
[04:34:54] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 34
[04:34:56] SMILES Parse Error: extra close parentheses while parsing: Cn1c(SC2=NCCN2C(=O)COC2OC(=O)NC2(O)CCCl)nnnc1-c1ccccc1)cc1OCCOCC1
[04:34:56] SMILES Parse Error: check for

[PPO Update 5/200] Avg episode reward (QED): 0.4043


[04:35:04] SMILES Parse Error: extra close parentheses while parsing: Cc1cc2c(NC(=O)c3c(Cl)cnsn3)nn(C(C)C)c2CC(C)C)o1
[04:35:04] SMILES Parse Error: check for mistakes around position 45:
[04:35:04] n3)nn(C(C)C)c2CC(C)C)o1
[04:35:04] ~~~~~~~~~~~~~~~~~~~~^
[04:35:04] SMILES Parse Error: Failed parsing SMILES 'Cc1cc2c(NC(=O)c3c(Cl)cnsn3)nn(C(C)C)c2CC(C)C)o1' for input: 'Cc1cc2c(NC(=O)c3c(Cl)cnsn3)nn(C(C)C)c2CC(C)C)o1'
[04:35:05] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 19
[04:35:06] Can't kekulize mol.  Unkekulized atoms: 10 11 12 13 15 16 22 23 24
[04:35:10] SMILES Parse Error: unclosed ring for input: 'Cc1ccc(C(=O)Nc2cccc(C[n+]3ccco4)o2)cc1C#N'
[04:35:11] Can't kekulize mol.  Unkekulized atoms: 1 3 4 5 8
[04:35:16] SMILES Parse Error: extra close parentheses while parsing: COc1ccc2oc(=O)ccc2c1)C(CF)(Cl)Cl
[04:35:16] SMILES Parse Error: check for mistakes around position 21:
[04:35:16] COc1ccc2oc(=O)ccc2c1)C(CF)(Cl)Cl
[04:35:16] ~~~~~~~~~~~~~~~~~~~~^
[04:35:16] SMILES Parse Error

[PPO Update 10/200] Avg episode reward (QED): 0.3883


...

[PPO Update 190/200] Avg episode reward (QED): 0.7456
[PPO Update 195/200] Avg episode reward (QED): 0.7815
[PPO Update 200/200] Avg episode reward (QED): 0.8023


[04:45:59] SMILES Parse Error: extra open parentheses while parsing: COc1cccc(Cl
[04:45:59] SMILES Parse Error: check for mistakes around position 9:
[04:45:59] COc1cccc(Cl
[04:45:59] ~~~~~~~~^
[04:45:59] SMILES Parse Error: Failed parsing SMILES 'COc1cccc(Cl' for input: 'COc1cccc(Cl'


  [Eval @ Update 200] Avg QED: 0.7987, Validity: 99.61%, Novelty: 1.95%
  Saved PPO actor-critic to checkpoints/smiles_actor_critic_ppo.pt


[04:46:05] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6 7 9
[04:46:09] SMILES Parse Error: syntax error while parsing: COc1cccc(Fl)c1CCNCCCC
[04:46:09] SMILES Parse Error: check for mistakes around position 11:
[04:46:09] COc1cccc(Fl)c1CCNCCCC
[04:46:09] ~~~~~~~~~~^
[04:46:09] SMILES Parse Error: extra open parentheses while parsing: COc1cccc(Fl)c1CCNCCCC
[04:46:09] SMILES Parse Error: check for mistakes around position 9:
[04:46:09] COc1cccc(Fl)c1CCNCCCC
[04:46:09] ~~~~~~~~^
[04:46:09] SMILES Parse Error: Failed parsing SMILES 'COc1cccc(Fl)c1CCNCCCC' for input: 'COc1cccc(Fl)c1CCNCCCC'


Final PPO policy metrics:
{'avg_qed': 0.7982828592004247, 'validity': 0.99609375, 'novelty': 0.017578125}


In [43]:
import torch
# torch.backends.cudnn.enabled = False

import importlib
import eval_utils
importlib.reload(eval_utils)

from smiles_data import create_smiles_dataloader
from smiles_model import SmilesRNNPolicy
from smiles_PPO import SmilesActorCritic
from eval_utils import (
    load_train_smiles_set,
    smiles_list_to_set,
    sample_random_from_dataset,
    sample_from_rnn_policy,
    sample_from_actor_critic,
    compute_metrics_from_smiles,
    compute_dataset_qed_stats,
    get_top_k_novel_molecules_unique,
    print_metrics_table,
)


def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_smiles_list = load_train_smiles_set(
        "data/guacamol_v1_all.smiles",
        max_size=200000,
    )
    train_smiles_set = smiles_list_to_set(train_smiles_list)
    print(f"Train SMILES subset size: {len(train_smiles_list)}, "
          f"canonical set size: {len(train_smiles_set)}")

    dataloader, vocab = create_smiles_dataloader(
        "data/guacamol_v1_all.smiles",
        batch_size=128,
        max_len=120,
    )

    pretrained_policy = SmilesRNNPolicy(
        vocab_size=len(vocab),
        embed_dim=256,
        hidden_dim=512,
        num_layers=2,
        dropout=0.1,
    ).to(device)
    pretrained_ckpt = "checkpoints/smiles_rnn_pretrained.pt"
    pretrained_policy.load_state_dict(torch.load(pretrained_ckpt, map_location=device))
    print(f"Loaded pretrained RNN policy from {pretrained_ckpt}")

    reinforce_policy = SmilesRNNPolicy(
        vocab_size=len(vocab),
        embed_dim=256,
        hidden_dim=512,
        num_layers=2,
        dropout=0.1,
    ).to(device)
    reinforce_ckpt = "checkpoints/smiles_policy_reinforce.pt"
    reinforce_policy.load_state_dict(torch.load(reinforce_ckpt, map_location=device))
    print(f"Loaded REINFORCE-tuned policy from {reinforce_ckpt}")

    actor_critic = SmilesActorCritic(
        vocab_size=len(vocab),
        embed_dim=256,
        hidden_dim=512,
        num_layers=2,
        dropout=0.1,
    ).to(device)
    ppo_ckpt = "checkpoints/smiles_actor_critic_ppo.pt"
    actor_critic.load_state_dict(torch.load(ppo_ckpt, map_location=device))
    print(f"Loaded PPO actor-critic from {ppo_ckpt}")

    num_eval_samples = 512

    dataset_samples = sample_random_from_dataset(
        train_smiles_list,
        num_samples=num_eval_samples,
    )
    metrics_dataset = compute_metrics_from_smiles(
        dataset_samples,
        train_smiles_set=train_smiles_set,
    )

    # RNN pretrained policy
    samples_pretrained = sample_from_rnn_policy(
        pretrained_policy,
        vocab=vocab,
        num_samples=num_eval_samples,
        max_len=120,
        greedy=False,
        device=device,
    )
    metrics_pretrained = compute_metrics_from_smiles(
        samples_pretrained,
        train_smiles_set=train_smiles_set,
    )

    # REINFORCE policy
    samples_reinforce = sample_from_rnn_policy(
        reinforce_policy,
        vocab=vocab,
        num_samples=num_eval_samples,
        max_len=120,
        greedy=False,
        device=device,
    )
    metrics_reinforce = compute_metrics_from_smiles(
        samples_reinforce,
        train_smiles_set=train_smiles_set,
    )

    # PPO actor-critic
    samples_ppo = sample_from_actor_critic(
        actor_critic,
        vocab=vocab,
        num_samples=num_eval_samples,
        max_len=120,
        greedy=False,
        device=device,
    )
    metrics_ppo = compute_metrics_from_smiles(
        samples_ppo,
        train_smiles_set=train_smiles_set,
    )

    results = {
        "dataset_random": metrics_dataset,
        "pretrained_rnn": metrics_pretrained,
        "reinforce": metrics_reinforce,
        "ppo": metrics_ppo,
    }
    print("\n=== Model Comparison ===")
    print_metrics_table(results)

    # dataset-wide QED
    dataset_qed_stats = compute_dataset_qed_stats(
        train_smiles_list,
        max_eval_size=50000,
    )
    print("\nDataset QED stats (subset):", dataset_qed_stats)

    # top k molecules
    top_k = get_top_k_novel_molecules_unique(
        smiles_list=samples_ppo,
        train_smiles_set=train_smiles_set,
        k=20,
    )
    print("\n=== Top-20 High-QED Novel Molecules from PPO ===")
    for i, r in enumerate(top_k, start=1):
        print(
            f"[{i:02d}] QED={r['qed']:.4f} | "
            f"SMILES={r['smiles']}"
        )


if __name__ == "__main__":
    main()


Train SMILES subset size: 200000, canonical set size: 199994
Loaded pretrained RNN policy from checkpoints/smiles_rnn_pretrained.pt
Loaded REINFORCE-tuned policy from checkpoints/smiles_policy_reinforce.pt
Loaded PPO actor-critic from checkpoints/smiles_actor_critic_ppo.pt


[05:56:39] Can't kekulize mol.  Unkekulized atoms: 1 2 11 12 13 14 15
[05:56:39] Can't kekulize mol.  Unkekulized atoms: 24 25 26 27 36
[05:56:39] Can't kekulize mol.  Unkekulized atoms: 5 6 7 10 30
[05:56:39] SMILES Parse Error: unclosed ring for input: 'c1ncsc1CN1CCN(C23CN4CCC(CC2)C4)CC1'
[05:56:39] Explicit valence for atom # 24 C, 6, is greater than permitted
[05:56:39] Can't kekulize mol.  Unkekulized atoms: 2 16 17 18 19 20 21
[05:56:39] SMILES Parse Error: extra open parentheses while parsing: COc1ccc(C2N(OCc3cccs3)C(=O)Nc3c2c(=O)oc2c3cc(OC)c(OC)cc2nc1O
[05:56:39] SMILES Parse Error: check for mistakes around position 8:
[05:56:39] COc1ccc(C2N(OCc3cccs3)C(=O)Nc3c2c(=O)oc2c
[05:56:39] ~~~~~~~^
[05:56:39] SMILES Parse Error: Failed parsing SMILES 'COc1ccc(C2N(OCc3cccs3)C(=O)Nc3c2c(=O)oc2c3cc(OC)c(OC)cc2nc1O' for input: 'COc1ccc(C2N(OCc3cccs3)C(=O)Nc3c2c(=O)oc2c3cc(OC)c(OC)cc2nc1O'
[05:56:39] SMILES Parse Error: extra close parentheses while parsing: Cc1nn(-c2cc(NC(=O)Nc3ccc(C(C)(C


=== Model Comparison ===
Model              |  Avg QED |  Validity |   Novelty
-----------------------------------------------------
dataset_random     |   0.5683 |   100.00% |     0.00%
pretrained_rnn     |   0.5084 |    89.84% |    89.65%
reinforce          |   0.7697 |    97.85% |    96.88%
ppo                |   0.8008 |   100.00% |   100.00%

Dataset QED stats (subset): {'avg_qed': 0.5536861915983652, 'top_qed': 0.9478732361077379}

=== Top-20 High-QED Novel Molecules from PPO ===
[01] QED=0.8118 | SMILES=CCOc1cccc(Cl)c1CCNC
[02] QED=0.8023 | SMILES=COc1cccc(Cl)c1CCNC
[03] QED=0.7804 | SMILES=COc1cccc(Cl)c1CCN
[04] QED=0.7786 | SMILES=COc1cccc(C#N)c1CCNC
[05] QED=0.7594 | SMILES=COc1cccc(Cl)c1CCNCC
[06] QED=0.7559 | SMILES=COc1cccc(Cl)c1CCNS
[07] QED=0.7551 | SMILES=COc1cccc(Cl)c1CCNCCC
[08] QED=0.6766 | SMILES=COc1cccc(Cl)1CCNC
[09] QED=0.3487 | SMILES=[O-]


In [2]:
import torch

from eval_utils import (
    load_train_smiles_set,
    smiles_list_to_set,
    sample_random_from_dataset,
    sample_from_rnn_policy,
    sample_from_actor_critic,
)
from smiles_data import create_smiles_dataloader
from smiles_model import SmilesRNNPolicy
from smiles_PPO import SmilesActorCritic
from viz_utils import (
    plot_qed_histograms,
    plot_metric_bars,
    compute_and_print_metrics,
    draw_top_molecules_grid,
    get_top_k_novel_molecules,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_smiles_list = load_train_smiles_set("data/guacamol_v1_all.smiles", max_size=200000)
train_smiles_set = smiles_list_to_set(train_smiles_list)

_, vocab = create_smiles_dataloader("data/guacamol_v1_all.smiles", batch_size=128, max_len=120)

# RNN
pretrained_policy = SmilesRNNPolicy(
    vocab_size=len(vocab),
    embed_dim=256,
    hidden_dim=512,
    num_layers=2,
    dropout=0.1,
).to(device)
pretrained_policy.load_state_dict(torch.load("checkpoints/smiles_rnn_pretrained.pt", map_location=device))

# REINFORCE
reinforce_policy = SmilesRNNPolicy(
    vocab_size=len(vocab),
    embed_dim=256,
    hidden_dim=512,
    num_layers=2,
    dropout=0.1,
).to(device)
reinforce_policy.load_state_dict(torch.load("checkpoints/smiles_policy_reinforce.pt", map_location=device))

# PPO
actor_critic = SmilesActorCritic(
    vocab_size=len(vocab),
    embed_dim=256,
    hidden_dim=512,
    num_layers=2,
    dropout=0.1,
).to(device)
actor_critic.load_state_dict(torch.load("checkpoints/smiles_actor_critic_ppo.pt", map_location=device))

num_eval_samples = 512

dataset_samples = sample_random_from_dataset(train_smiles_list, num_samples=num_eval_samples)

from eval_utils import sample_from_rnn_policy, sample_from_actor_critic

samples_pretrained = sample_from_rnn_policy(
    pretrained_policy, vocab=vocab, num_samples=num_eval_samples, device=device
)
samples_reinforce = sample_from_rnn_policy(
    reinforce_policy, vocab=vocab, num_samples=num_eval_samples, device=device
)
samples_ppo = sample_from_actor_critic(
    actor_critic, vocab=vocab, num_samples=num_eval_samples, device=device
)

from viz_utils import plot_qed_histograms

model_smiles_dict = {
    "dataset_random": dataset_samples,
    "pretrained_rnn": samples_pretrained,
    "reinforce": samples_reinforce,
    "ppo": samples_ppo,
}

plot_qed_histograms(
    model_smiles_dict,
    bins=30,
    save_path="figures/qed_histogram.png",
)

from viz_utils import compute_and_print_metrics, plot_metric_bars

metrics_dict = compute_and_print_metrics(
    model_smiles_dict,
    train_smiles_set=train_smiles_set,
)

plot_metric_bars(
    metrics_dict,
    save_path="figures/model_metrics_bar.png",
)

top_k = get_top_k_novel_molecules(
    smiles_list=samples_ppo,
    train_smiles_set=train_smiles_set,
    k=9,
)

smiles_top9 = [r["smiles"] for r in top_k]
qeds_top9 = [r["qed"] for r in top_k]

draw_top_molecules_grid(
    smiles_top9,
    qeds_top9,
    n_cols=3,
    sub_img_size=(300, 300),
    save_path="figures/top9_qed_molecules.png",
)


ModuleNotFoundError: No module named 'rdkit'